# 0. Preparation

In [ ]:
#In order to acquire the dataset in a Cloud VM environment

'''
from google.cloud import storage

bucket_name = "isic-bucket"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

my_prefix = "Training/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine

import zipfile
with zipfile.ZipFile('./Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('./lol')
        '''

In [1]:
import os
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#from glob import glob
#import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score

import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.layers.normalization import BatchNormalization
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras import backend as K 


2022-06-26 15:47:36.805815: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-26 15:47:36.805849: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# 1. Acquiring the training data

## 1.1 Loading - splitting the data

In [3]:
import PIL
train_folder = './Training'
valid_folder = './Validation'

train_data = pd.read_csv(train_folder + '/' + 'ISIC2018_Training_GroundTruth.csv')
valid_data = pd.read_csv(valid_folder + '/' + 'ISIC2018_Validation_GroundTruth.csv')

#print(len(benign))
#print(len(malignant))


In [4]:
def load_images(load_dir, image_list):
    lst = [np.asarray(Image.open(load_dir + '/' + filename + '.jpg').convert('RGB')) for filename in image_list]
    return lst

In [8]:
def load_prepare_dataset(dataset_csv, folder):

    malignant = train_data.loc[(train_data['MEL'] == 1)| (train_data['BCC'] == 1)]
    benign = train_data.loc[(train_data['MEL'] == 0) & (train_data['BCC'] == 0)]
    
    # Load in training pictures 
    benign_images = load_images(folder, benign['image'])
    malignant_images = load_images(folder, malignant['image'])
    print(f'len malignant:{len(malignant)}')
    print(f'len benign:{len(benign)}')
    X_benign = np.array(benign_images, dtype='uint8')
    X_malignant = np.array(malignant_images, dtype='uint8')

    #####################################
    #data augmentation
    #####################################

    img_generator = ImageDataGenerator(horizontal_flip=True)
    img_generator = img_generator.flow(malignant_images, batch_size=len(malignant_images))
    batch = img_generator.next()

    for augm_image in batch:
        X_malignant.append(np.array(augm_image, dtype='uint8'))
    
    img_generator = ImageDataGenerator(vertical_flip=True)
    img_generator = img_generator.flow(malignant_images, batch_size=len(malignant_images))
    batch = img_generator.next()

    for augm_image in batch:
        X_malignant.append(np.array(augm_image, dtype='uint8'))

    print('After augmentation')
    print(f'len malignant:{X_malignant.shape[0]}')
    print(f'len benign:{X_benign.shape[0]}')

    # Create labels
    y_benign = np.zeros(X_benign.shape[0])
    y_malignant = np.ones(X_malignant.shape[0])

    # Merge data 
    x = np.concatenate((X_benign, X_malignant), axis = 0)
    y = np.concatenate((y_benign, y_malignant), axis = 0)

    # Shuffle data
    s = np.arange(x.shape[0])
    np.random.shuffle(s)
    x = x[s]
    y = y[s]

    y = to_categorical(y, num_classes= 2)
    x = x/255.

    return x, y
    

In [6]:
#train_folder = './small'
#benign = benign['image'].iloc[:40]
#malignant = malignant['image'].iloc[:40]

X_train, y_train = load_prepare_dataset(train_data, train_folder)

KeyboardInterrupt: 

## 1.2 Data augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale=(1/255.),shear_range = 0.2,zoom_range=0.2,
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory(directory = train_dir,target_size=(64,64),
                                                batch_size=32,
                                                class_mode = "binary")
test_datagen = ImageDataGenerator(rescale=(1/255.))
test_set = test_datagen.flow_from_directory(directory = test_dir,target_size=(64,64),
                                                batch_size=32,
                                                class_mode = "binary")

# 2. Data visualisation and exploration (work in progress)

## 2.1 Data exploration

In [ ]:
'''w=40
h=30
fig=plt.figure(figsize=(12, 8))
columns = 5
rows = 3

for i in range(1, columns*rows +1):
    ax = fig.add_subplot(rows, columns, i)
    if (y_train[i] == np.asarray([1,0])).all():
        ax.title.set_text('Benign')
    else:
        ax.title.set_text('Malignant')
    plt.imshow(X_train[i], interpolation='nearest')
plt.show()'''

## 2.2 Data visualisation

In [ ]:
#Missing graphs for presentation

# 3. Building the models 
## 3.1 Necessary functions

In [ ]:
def run_model(model):
    history = model.fit(X_train, y_train, validation_split=0.2,
                        epochs= epochs, batch_size= batch_size, verbose=0, 
                        callbacks=[learning_rate_reduction]
                    )
                    
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
def build_model(input_shape= (224,224,3), lr = 1e-3, num_classes= 2, init= 'normal', activ= 'relu', optim= 'adam'):
    
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3),padding = 'Same',input_shape=input_shape,
                     activation= activ, kernel_initializer='glorot_uniform'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(3, 3),padding = 'Same', 
                     activation =activ, kernel_initializer = 'glorot_uniform'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer=init))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()

    if optim == 'rmsprop':
        optimizer = RMSprop(lr=lr)

    else:
        optimizer = Adam(lr=lr)

    model.compile(optimizer = optimizer ,loss = "binary_crossentropy", metrics=["accuracy"])
    return model


In [ ]:
def save_model(model, model_filename):
    model_json = model.to_json()

    with open(model_filename + ".json", "w") as json_file:
        json_file.write(model_json)
        
    model.save_weights(model_filename + ".h5")
    print("Saved")

    del model
    K.clear_session()

## 3.2 Custom CNN

In [ ]:
input_shape = (224,224,3)
lr = 1e-5
init = 'normal'
activ = 'relu'
optim = 'adam' # <-----NEEDS TO CHANGE
epochs = 50
batch_size = 64

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=1e-7)

cnn_model = build_model(lr=lr, init= init, activ= activ, optim=optim, input_shape= input_shape)
run_model(cnn_model)

### 3.1.1 Cross-validation

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=17)

cvscores = []
for train, test in kfold.split(X_train, y_train):
    model = build_model(lr=lr, 
                  init= init, 
                  activ= activ, 
                  optim=optim, 
                  input_shape= input_shape)
    
    model.fit(X_train[train], y_train[train], epochs=epochs, batch_size=batch_size, verbose=0)
    scores = model.evaluate(X_train[test], y_train[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    K.clear_session()
    del model
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

### 3.1.2 Testing

In [ ]:
X_test, y_test = load_prepare_dataset(valid_data, valid_folder)

In [ ]:
model = build_model(lr=lr, 
              init= init, 
              activ= activ, 
              optim=optim, 
              input_shape= input_shape)

model.fit(X_train, y_train,
          epochs=epochs, batch_size= batch_size, verbose=0,
          callbacks=[learning_rate_reduction]
         )

y_pred = model.predict_classes(X_test)

print(accuracy_score(np.argmax(y_test, axis=1),y_pred))

save_model(model, 'cnn_model')

## 3.2 ResNet50
### 3.2.1 Building the model

In [ ]:
input_shape = (224,224,3)
lr = 1e-5
epochs = 50
batch_size = 64

model = ResNet50(include_top=True,
                 weights= None,
                 input_tensor=None,
                 input_shape=input_shape,
                 pooling='avg',
                 classes=2)


run_model(model)

### 3.2.2 Testing

In [ ]:
model.fit(X_train, y_train,
          epochs=epochs, batch_size= epochs, verbose=0,
          callbacks=[learning_rate_reduction]
         )

y_pred = model.predict(X_test)

print(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

save_model(model, 'resnet_model')

## 3.3 VGG19

In [ ]:
input_shape = (224,224,3)
lr = 1e-5
epochs = 10
batch_size = 256


vgg19_model = VGG16(include_top=False,
                 weights= 'imagenet',
                 input_tensor=None,
                 input_shape=input_shape,
                 pooling='avg',
                 classes=2)

for layer in vgg19_model.layers:
    layer.trainable = False
    
model = Sequential()
model.add(vgg19_model)
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))

run_model(model)

In [ ]:
model.fit(X_train, y_train,
          epochs=epochs, batch_size= epochs, verbose=0,
          callbacks=[learning_rate_reduction]
         )

y_pred = model.predict(X_test)

print(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

save_model(model, 'vgg_model')